<a href="https://colab.research.google.com/github/AstroBoy1/santander/blob/master/basic_stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification (with probabilities) + Detailed workflow
***

In [1]:
!pip3 uninstall xgboost
!pip3 install xgboost
!pip uninstall sklearn
!pip install sklearn
!pip install scikit-optimize
!pip install vecstack

Uninstalling xgboost-0.82:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/xgboost-0.82.dist-info/*
    /usr/local/lib/python3.6/dist-packages/xgboost/*
    /usr/local/xgboost/libxgboost.so
Proceed (y/n)? y
  Successfully uninstalled xgboost-0.82
  Using cached https://files.pythonhosted.org/packages/6a/49/7e10686647f741bd9c8918b0decdb94135b542fe372ca1100739b8529503/xgboost-0.82-py2.py3-none-manylinux1_x86_64.whl
Uninstalling sklearn-0.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/sklearn-0.0.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled sklearn-0.0


In [2]:
from google.colab import drive
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn import svm
from collections import Counter
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import time
import statistics
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier
from vecstack import stacking
from vecstack import StackingTransformer
#from skopt import BayesSearchCV

Using TensorFlow backend.


In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
train_fn = '/content/gdrive/My Drive/santander_data/train.csv'
valid_fn = '/content/gdrive/My Drive/santander_data/test.csv'
pred_fn = '/content/gdrive/My Drive/santander_data/submission12.csv'
train_data_df = pd.read_csv(train_fn)
test_data_df = pd.read_csv(valid_fn)
train_data_x = train_data_df.drop(columns=["ID_code", "target"]).values
train_data_y = train_data_df["target"].values
test_data_x = test_data_df.drop(columns=["ID_code"]).values

# Import

In [0]:
from glob import glob
import re 
import numpy as np
np.random.seed(0) # ensure reproducibility
np.set_printoptions(suppress = True)
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import log_loss
# Models
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# NN
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Stacking
from vecstack import stacking

In [0]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder

def auc(y_true, y_pred):
    """ROC AUC metric for both binary and multiclass classification.
    
    Parameters
    ----------
    y_true : 1d numpy array
        True class labels
    y_pred : 2d numpy array
        Predicted probabilities for each class
    """
    ohe = OneHotEncoder(sparse=False)
    y_true = ohe.fit_transform(y_true.reshape(-1, 1))
    auc_score = roc_auc_score(y_true, y_pred)
    return auc_score

# Prepare data

In [39]:
# n_classes = 3
n_classes = 2
# Create data: 500 example, 5 feature, 3 classes
# X, y = make_classification(n_samples=500, n_features=5, 
#                            n_informative=3, n_redundant=1, 
#                            n_classes=n_classes, flip_y=0, 
#                            random_state=0)
length = 10000
X, y = train_data_x[:length], train_data_y[:length]

# Make train/test split
# As usual in machine learning task we have X_train, y_train, and X_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print('Train shape:', X_train.shape)
print('Test shape: ', X_test.shape)

Train shape: (8000, 200)
Test shape:  (2000, 200)


# Initialize 1st level models

In [0]:
def build_keras_model_1():
    model = Sequential()
    model.add(Dense(64, 
                    input_dim=X_train.shape[1], 
                    kernel_initializer='normal', 
                    activation='relu'))
    model.add(Dense(n_classes, 
                    kernel_initializer='normal', 
                    activation='softmax'))
    model.compile(optimizer='rmsprop', 
                  loss='categorical_crossentropy', 
                  metrics=['categorical_accuracy'])
    return model

# Caution! All models and parameter values are just 
# demonstrational and shouldn't be considered as recommended.
models_1 = [ 
    GaussianNB(),
    
    LogisticRegression(random_state=0),
    
    ExtraTreesClassifier(random_state=0, n_jobs=-1, 
                         n_estimators=100, max_depth=3),
                         
    RandomForestClassifier(random_state=0, n_jobs=-1, 
                           n_estimators=100, max_depth=3),
        
    XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                  n_estimators=100, max_depth=3, categories='auto'),
                  
    LGBMClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                   n_estimators=100, max_depth=3),
                  
    #KerasClassifier(build_fn=build_keras_model_1, epochs=2, 
    #                batch_size=32, verbose=0)
]

# Perform stacking

In [16]:
save_directory = '/content/gdrive/My Drive/santander_results/'
S_train_1, S_test_1 = stacking(models_1,                   # list of models
                               X_train, y_train, X_test,   # data
                               regression=False,           # classification task (if you need 
                                                           #     regression - set to True)
                               mode='oof_pred',            # mode: oof for train set, fit on full 
                                                           #     train and predict test set once
                               needs_proba=True,           # predict probabilities (if you need 
                                                           #     class labels - set to False) 
                               ##save_dir='.',               # save result and log in current dir 
                               save_dir=save_directory,                                 #     (to disable saving - set to None)
                               metric=auc,            # metric: callable
                               n_folds=5,                  # number of folds
                               stratified=True,            # stratified split for folds
                               shuffle=True,               # shuffle the data
                               random_state=0,             # ensure reproducibility
                               verbose=2)                  # print all info

task:         [classification]
n_classes:    [2]
metric:       [auc]
mode:         [oof_pred]
n_models:     [6]

model  0:     [GaussianNB]
    fold  0:  [0.84649203]
    fold  1:  [0.86661607]
    fold  2:  [0.84802665]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

    fold  3:  [0.87653616]
    fold  4:  [0.88036613]
    ----
    MEAN:     [0.86360741] + [0.01409082]
    FULL:     [0.86311663]

    Fitting on full train set...

model  1:     [LogisticRegression]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, ma

    fold  0:  [0.80026431]


...

    fold  2:  [0.79150793]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


    fold  3:  [0.78760355]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

    fold  4:  [0.81427279]
    ----
    MEAN:     [0.79879043] + [0.01195830]
    FULL:     [0.79849410]

    Fitting on full train set...

Result was saved to [/content/gdrive/My Drive/santander_results/[2019.03.26].[04.42.41].477166.c4aa2f.npy]


# Look at the result

In [17]:
print('We have %d classes and %d models so in resulting arrays \
we expect to see %d columns.' % (n_classes, len(models_1), n_classes * len(models_1)))
print('S_train_1 shape:', S_train_1.shape)
print('S_test_1 shape: ', S_test_1.shape)

We have 2 classes and 6 models so in resulting arrays we expect to see 12 columns.
S_train_1 shape: (8000, 12)
S_test_1 shape:  (2000, 12)


In [34]:
S_train_1[:2]

array([[0.88595608, 0.11404392, 0.87512032, 0.12487968, 0.90076603,
        0.09923397, 0.91364649, 0.08635351, 0.86732459, 0.13267542,
        0.91284642, 0.08715358],
       [0.99596026, 0.00403974, 0.98994982, 0.01005018, 0.90782715,
        0.09217285, 0.91324549, 0.08675451, 0.96264553, 0.03735447,
        0.96822305, 0.03177695]])

In [35]:
S_test_1[:2]

array([[0.96200187, 0.03799813, 0.94323898, 0.05676102, 0.89576226,
        0.10423774, 0.89789474, 0.10210526, 0.92684668, 0.07315334,
        0.93933053, 0.06066947],
       [0.99292921, 0.00707079, 0.9939683 , 0.0060317 , 0.90598083,
        0.09401917, 0.90470871, 0.09529129, 0.95539749, 0.04460249,
        0.95137477, 0.04862523]])

In [38]:
S_train_1[0][0], S_train_1[0][1], S_train_1[0][1]

(0.8859560775904103, 0.11404392240959342, 0.11404392240959342)

# Our arrays and log were saved in current dir

In [20]:
names = sorted(glob(save_directory + '*.npy'))
npy_1_name = names[0] # for later use

print('Arrays:')
for name in names:
    print(name)

names = sorted(glob(save_directory + '*.log.txt'))
log_1_name = names[0] # for later use

print('\nLogs:')
for name in names:
    print(name)

Arrays:
/content/gdrive/My Drive/santander_results/[2019.03.26].[04.42.41].477166.c4aa2f.npy

Logs:
/content/gdrive/My Drive/santander_results/[2019.03.26].[04.42.41].477166.c4aa2f.log.txt


# Time to collect results

After several (many) days of building, optimizing, and testing models we have a lot of files with saved OOF.  
At this point we can load and use OOF of specific model or all OOF we have.

# Find specific model

We can open logs and find the model of interest.  
We can do it programmatically or just open logs in editor.  
Name of the `.log.txt` file is the same as the name of corresponding `.npy` file (except extension).  
To find columns containing OOF of specific model we use model index from log:
* if we predicted class labels - corresponding column index is just model index
* if we predicted probabilities - corresponding column index is model index multiplied by number of classes

In [0]:
print("Let's open this log: %s" % log_1_name)
with open(log_1_name) as f:
    lines = f.readlines()

print("Let's look what models did we build in those session.\n")
for line in lines:
    if re.search(r'^model [0-9]+', line):
        print(line)

Let's open this log: [2018.02.01].[15.41.41].305268.0eadc0.log.txt
Let's look what models did we build in those session.

model 0:    [GaussianNB]

model 1:    [LogisticRegression]

model 2:    [ExtraTreesClassifier]

model 3:    [RandomForestClassifier]

model 4:    [XGBClassifier]

model 5:    [LGBMClassifier]

model 6:    [KerasClassifier]



# Load specific model OOF

Let's say we are interested in `LGBMClassifier`.  
We found out that it has index 5.  
Then we load target `.npy` file and because of probabilities we need 3 columns from 15 (5 \* 3) to 18 (5 \* 3 + 3)

In [28]:
print("Let's load this .npy file: %s" % npy_1_name)
S = np.load(npy_1_name)
S_train_lgbm = S[0][:, 15:18]
S_test_lgbm = S[1][:, 15:18]

Let's load this .npy file: /content/gdrive/My Drive/santander_results/[2019.03.26].[04.42.41].477166.c4aa2f.npy


In [0]:
S_train_lgbm[:5]

array([[ 0.00040829,  0.00281319,  0.99677852],
       [ 0.99732125,  0.00258249,  0.00009626],
       [ 0.98322854,  0.01610955,  0.00066191],
       [ 0.00107737,  0.99633895,  0.00258368],
       [ 0.97101719,  0.02843959,  0.00054321]])

In [0]:
S_test_lgbm[:5]

array([[ 0.60639131,  0.3588515 ,  0.03475718],
       [ 0.03609523,  0.90174785,  0.06215692],
       [ 0.08650007,  0.89717473,  0.0163252 ],
       [ 0.00068572,  0.98858075,  0.01073353],
       [ 0.00122693,  0.99814513,  0.00062793]])

# Compute score of specific model

In [0]:
print('LGBMCLassifier log loss: %.8f' % log_loss(y_train, S_train_lgbm))

LGBMCLassifier log loss: 0.41430248


# Load ALL OOF

***Note:*** If you load OOF from scratch, don't forget to load `y_train` from initial dataset too.

In [0]:
print('We have %d classes and %d models TOTAL so in resulting arrays \
we expect to see %d columns.' % (n_classes, len(models_1) + len(models_2), 
                                 n_classes * (len(models_1) + len(models_2))))

We have 3 classes and 8 models TOTAL so in resulting arrays we expect to see 24 columns.


In [42]:
# Create empty arrays
S_train_all = np.zeros((X_train.shape[0], 0))
S_test_all = np.zeros((X_test.shape[0], 0))

# Load results
for name in sorted(glob(save_directory + '*.npy')):
    print('Loading: %s' % name)
    S = np.load(name)
    S_train_all = np.c_[S_train_all, S[0]]
    S_test_all = np.c_[S_test_all, S[1]]
    
print('\nS_train_all shape:', S_train_all.shape)
print('S_test_all shape: ', S_test_all.shape)

Loading: /content/gdrive/My Drive/santander_results/[2019.03.26].[04.42.41].477166.c4aa2f.npy

S_train_all shape: (8000, 12)
S_test_all shape:  (2000, 12)


# Apply 2nd level model

Just use class 0 probability

In [59]:
columns = [n for n in range(0, S_train_all.shape[1], 2)]
l2_train = S_train_all[:, columns]
l2_test = S_test_all[:, columns]
l2_train.shape, l2_test.shape

((8000, 6), (2000, 6))

In [60]:
# Initialize 2nd level model
model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                      n_estimators=100, max_depth=3)
    

# Fit 2nd level model
# model = model.fit(S_train_all, y_train)
model = model.fit(l2_train, y_train)


# Predict
# y_pred = model.predict_proba(S_test_all)
y_pred = model.predict_proba(l2_test)

from sklearn.metrics import roc_auc_score

# Final prediction score
# print('Final prediction score: %.8f' % log_loss(y_test, y_pred))
y_pred_final = [elem[1] for elem in y_pred]
roc_auc_score(y_test, y_pred_final)

0.8461037566493023

No change with redundant features

In [61]:
# Initialize 2nd level model
model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                      n_estimators=100, max_depth=3)
    

# Fit 2nd level model
model = model.fit(S_train_all, y_train)
#model = model.fit(l2_train, y_train)


# Predict
y_pred = model.predict_proba(S_test_all)
#y_pred = model.predict_proba(l2_test)

from sklearn.metrics import roc_auc_score

# Final prediction score
# print('Final prediction score: %.8f' % log_loss(y_test, y_pred))
y_pred_final = [elem[1] for elem in y_pred]
roc_auc_score(y_test, y_pred_final)

0.8461037566493023